In [1]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#########################################這區是告訴程式那些可行的Xpath######################################################
#Xpath 可以多用contains的方式來寫，並用or增加邏輯覆蓋率，如果在102-104行都顯示找不到XX，就可以在這邊手動增加，提高找尋到內容的機率

author_element_possible_xpath_list=['//*[@id="interview_prof"]/p[2]/strong']
article_element_possible_xpath_list = ['//*[@id="detail_contents"]//p']
title_element_possible_xpath_list = ['//*[@id="main"]/dl[1]/dt']


###########################################################################################################################

#CNBC的TRENDING 只會列出五篇
max_number_article = 5
main_driver = webdriver.Chrome('C:\chromedriver.exe')  # 注意你們放CHROMEDRIVER的位置
main_driver.implicitly_wait(30)
main_driver.get('https://www.nippon.com/hk/')
#假裝滑動到該位置，讓javascript自動生成 (這個方式無法在headless的chrome內完成)
main_driver.execute_script("window.scrollTo(0, document.body.scrollHeight/9)")
#等待javascript生成完成
time.sleep(5)
#直接用Xpath取得文章標題
article_title_list=[]
address_list=[]

#要注意瀏覽器不要動它!不要更改大小!不然javascript又會再升成不同的Xpath，這樣17行也要更改才行!
for x in range(max_number_article):
    if((x+1)%2 == 1):
        num = 2
    else:
        num = 1
    tmp = main_driver.find_element_by_xpath('//*[@id="top_tab_wrp"]/div[3]/div['+str(x+1)+']/dl/dd/p['+str(num)+']/a')
    #//*[@id="top_tab_wrp"]/div[3]/div[1]/dl/dd/p[2]/a
    #//*[@id="top_tab_wrp"]/div[3]/div[2]/dl/dd/p[1]/a
    #//*[@id="top_tab_wrp"]/div[3]/div[3]/dl/dd/p[2]/a
    #//*[@id="top_tab_wrp"]/div[3]/div[4]/dl/dd/p[1]/a
    #//*[@id="top_tab_wrp"]/div[3]/div[5]/dl/dd/p[2]/a
    article_title_list.append(tmp)
    address_list.append(tmp.get_attribute('href'))
    print(address_list[x])
    #注意我method內爬文的寫法，分成三部分，1找分類 2找作者 3找內文
def Retrive_CNBC_Content(address):
    driver = webdriver.Chrome('C:\chromedriver.exe') 
    driver.implicitly_wait(30)
    driver.get(address)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/6)")
    
    author_logged=""
    article_content_logged=""
    title_logged=""
    
    author_content=""
    article_content=""
    title_content=""
    
    try:
        #找作者#######################################################################
        for xpath in author_element_possible_xpath_list: 
            author_elements_found_list = driver.find_elements_by_xpath(xpath)
            if(len(author_elements_found_list)>0):
                author_logged = "  找到作者"
                author_content = [t.text for t in author_elements_found_list] # return strings
                author_content = "/".join(author_content)
                #print(',作者是: '+author_content)#把作者呈現出來
                break
            else: author_logged = "  找不到作者"
        
        #找內文#######################################################################
        for xpath in article_element_possible_xpath_list:          
            article_elements_found_list = driver.find_elements_by_xpath(xpath)
            if(len(article_elements_found_list)>0):
                article_content_logged = "  找到文章"
                article_content = [t.text for t in article_elements_found_list] # return strings
                article_content = "".join(article_content)
                #print(article_content)#把文章呈現出來
                #print(len(article_content))#對於一些特殊版面的文章找不到就會是0
                break
            else: article_content_logged = "  找不到文章"
        #找標題#############################################################################
        for xpath in title_element_possible_xpath_list:          
            title_element_found_list = driver.find_elements_by_xpath(xpath)
            if(len(title_element_found_list)>0):
                title_logged = "  找到標題"
                title_content = [t.text for t in title_element_found_list] # return strings
                title_content = "".join(title_content)
                break
            else: title_logged = "  找不到標題"
    except NoSuchElementException:  #這個例外是要處理另一種排版的畫面，RETAIL類別的文章會有兩種排版畫面，因此兩種Xpath
        print("出現找不到element的例外!")#印出結果方便分析，如果遇到找不到內容的網站，就要點進連結去看是不是
    print('搜尋狀況: ')
    print(author_logged)
    print(article_content_logged)
    print(title_logged)
    
    print('標題: '+title_content)
    print('作者: '+author_content)
    print('內容:' +article_content)
    print("="*130)
    driver.close()
    
for address in address_list:
    print('文章網址: '+address)
    time.sleep(5)#休息五秒不要讓對方網站發現你是機器人一直開她的網頁
    Retrive_CNBC_Content(address)
        
main_driver.close()

https://www.nippon.com/hk/views/b06007/
https://www.nippon.com/hk/currents/d00451/
https://www.nippon.com/hk/in-depth/a03801/
https://www.nippon.com/hk/currents/d00426/
https://www.nippon.com/hk/features/h00208/
文章網址: https://www.nippon.com/hk/views/b06007/
搜尋狀況: 
  找不到作者
  找到文章
  找到標題
標題: 以柚子振興農村：人口900人年收30億日圓的村莊
作者: 
內容:從四國高知機場出發，遠眺右手邊湛藍色的太平洋，車程約兩小時，往深山裡去，便能抵達以生產柚子聞名的馬路村。高知縣柚子生產量佔全日本五成，堪稱日本第一。其中馬路村使用有機柚子開發新商品，是將柚子魅力推廣至全國的先驅。「日本擁有世上前所未有的許多不同的香酸柑橘種類。如柚子、酢橘、臭橙、酸桔、蛇腹、金柑、香橙、佛手柑、晚白柚等，日本是真正的柑橘王國。」馬路村農協工會東谷望史會長充滿自信地說道。從前食用秋刀魚或松茸時，很多地區會搭配酢橘來使用，而椪醋中使用臭橙或香橙的地區也不少，柚子的存在感變得相對薄弱。「柚子是代表日本的香酸柑橘之一，自古以來柚子皮多為增加香氣之用。然而，從料理調味至甜點製作，這數十年來持續發展，不僅果皮，甚至果汁、果肉、種子等皆被廣泛使用。」東谷先生說道。馬路村「柚子之森」加工廠入口。提供入廠參觀。「即使從小常吃柚子仍不覺得膩。對我而言，我有堅定不移的自信能讓柚子推廣到日本全國各地。」東谷先生回顧。栽培有著大大的刺、且具有直挺挺生長等強烈特性的柚子，需花費不少功夫。並補充說道，「自古便有這樣的說法『桃栗3年、柿子8年、柚子大塊頭則得花上18年』表示水果長成所需的年數。」在馬路村，種植的大多是在歐洲有野生柚之稱的「實生柚」。與一般以接木方式種植，可以早些收成的柚子相比，需要較長時間培育的實生柚多了股酸味與清香，在廚師間具有相當高的人氣。
文章網址: https://www.nippon.com/hk/currents/d00451/
搜尋狀況: 
  找到作者
  找到文章
  找到標題
標題: 原日本銀行總裁白川方明，對